# 导入库


In [ ]:
import pandas as pd
import numpy as np

df_kl=pd.read_excel('~/Downloads/kl-1.xlsx')
df_kl.insert(loc=0,column='序号',value = [1,5,6,2,4,3])
df_kl_t=df_kl.sort_values(by=['序号'],ascending=True)
page_2 = df_kl_t[['项目','日均客流','同期日均客流','日均客流同比','平日日均客流','周末日均客流','周末/平日比','同期周末/平日比']]
page_2.astype({'周末/平日比':'float','同期周末/平日比':'float'})
page_2['周末占比同比'] = df_kl_t['周末/平日比'] - df_kl_t['同期周末/平日比']
page_2 = df_kl_t[['项目','日均客流','同期日均客流','日均客流同比','平日日均客流','周末日均客流','周末/平日比','周末占比同比']]

df_xs=pd.read_excel('~/Downloads/xs-1.xlsx')
df_all = df_xs.loc[(df_xs.项目分类 =='项目整体')]
df_all.insert(loc=0,column='序号',value =[1,5,6,2,4,3])
df_xs_a=df_all.sort_values(by=['序号'],ascending=True)
page_3 = df_xs_a[['项目','日均销售额','同期日均销售额','日均销额售同比','平日日均销售额','周末日均销售额','周末/平日比','周末占比同比']]


df_major = df_xs.loc[(df_xs.项目分类 =='主次力店')]
df_major.insert(loc=0,column='序号',value =[1,5,6,2,4,3])
df_xs_m=df_major.sort_values(by=['序号'],ascending=True)
page_4 = df_xs_m[['项目','日均销售额','同期日均销售额','日均销额售同比','平日日均销售额','周末日均销售额','周末/平日比','周末占比同比']]


df_normal = df_xs.loc[(df_xs.项目分类 =='一般店铺')]
df_normal.insert(loc=0,column='序号',value =[1,5,6,2,4,3])
df_xs_n=df_normal.sort_values(by=['序号'],ascending=True)
page_5 = df_xs_n[['项目','日均销售额','同期日均销售额','日均销额售同比','平日日均销售额','周末日均销售额','周末/平日比','周末占比同比']]



page_1 = pd.merge(page_2,page_3,on='项目',how='left')[['项目','日均客流','日均客流同比','日均销售额','日均销额售同比']]


rjkl=page_2.日均客流.sum()/6
rjkl_tq = page_2.同期日均客流.sum()
rjkl_tb = (rjkl-rjkl_tq)/rjkl_tq

rjxs=page_3.日均销售额.sum()/6
rjxs_tq=page_3.同期日均销售额.sum()
rjxs_tb=(rjxs-rjxs_tq)/rjxs_tq

df_a_row1 = pd.DataFrame(np.insert(page_1.values,0,values=['均值',rjkl_tq,rjkl_tb,rjxs_tq,rjxs_tb],axis=0))

page_2=page_2.round(decimals=1)
page_3=page_3.round(decimals=0)
page_4=page_4.round(decimals=0)
page_5=page_5.round(decimals=0)
page_1=page_1.round(decimals=0)

with pd.ExcelWriter('~/Downloads/ppt.xlsx') as writer:
    df_a_row1.to_excel(writer,sheet_name='page_1')
    page_2.to_excel(writer,sheet_name='page_2')
    page_3.to_excel(writer,sheet_name='page_3')
    page_4.to_excel(writer,sheet_name='page_4')
    page_5.to_excel(writer,sheet_name='page_5')
    


In [ ]:
import pandas as pd
import numpy as np

df_1 = pd.read_excel('~/Downloads/南京弘阳广场客流202105.xls',header=5)
df_ps= df_1.loc[(df_1.属性 =='平时')][['日期','星期','属性','P00011']].sort_values(by='日期').rename(columns={'P00011':'客流'})
df_zm= df_1.loc[(df_1.属性 =='周末')][['日期','星期','属性','P00011']].sort_values(by='日期').rename(columns={'P00011':'客流'})

df_ps=df_ps.groupby(df_ps.属性).agg({'客流':'sum'}).reset_index()
df_zm=df_zm.groupby(df_zm.属性).agg({'客流':'sum'}).reset_index()
df_ps 

In [ ]:
import pandas as pd
import numpy as np
from output_style_format import to_excel_auto_column_weight

def get_df(folderpath,shoppingmall,month:str):
    #folderpath,month1,month2,shoppingmall,form
    df_xs_all = pd.read_excel(folderpath+shoppingmall+month+'.xlsx')
    df_ppk = pd.read_excel(folderpath+'/品牌库.xlsx')


    df_xs_all =df_xs_all[['项目名称','铺位类型','品牌名称','营业额','营业额日期']]
    df_ppk = df_ppk[['品牌名称','业态']]

    df_xs_all_ppk = pd.merge(df_xs_all,df_ppk,on='品牌名称',how='left')

    df_xs_zcld = df_xs_all_ppk.loc[(df_xs_all_ppk.业态=='主次力店')]
    df_xs_zcld = df_xs_zcld.groupby(df_xs_zcld.营业额日期).agg({'营业额':'sum'}).reset_index()

    df_xs_ybdp = df_xs_all_ppk.loc[(df_xs_all_ppk.业态!='主次力店')]
    df_xs_ybdp = df_xs_ybdp.groupby(df_xs_ybdp.营业额日期).agg({'营业额':'sum'}).reset_index()

    df_xs_zcld['营业额日期']=df_xs_zcld['营业额日期'].apply(lambda x : x.strftime('%Y-%m-%d'))
    df_xs_ybdp['营业额日期']=df_xs_ybdp['营业额日期'].apply(lambda x : x.strftime('%Y-%m-%d'))

    return df_xs_zcld,df_xs_ybdp

df1,df2=get_df('~/Desktop/销售客流pandas/','合肥弘阳广场','202105')
df5,df6=get_df('~/Desktop/销售客流pandas/','合肥弘阳广场','202205')

with pd.ExcelWriter('~/Downloads/xs_合肥弘阳广场1.xlsx',engine="openpyxl") as writer:
    to_excel_auto_column_weight(df1,writer,'主次力店202105')
    to_excel_auto_column_weight(df2,writer,'一般店铺202105')
    to_excel_auto_column_weight(df5,writer,'主次力店202205')
    to_excel_auto_column_weight(df6,writer,'一般店铺202205')



In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime


def get_diff_xs(folderpath,month1,month2,shoppingmall,form):
    """
    Parameters: Str: month1,month2,shoppingmall
    get_diff_xs('~/Desktop/销售客流pandas/','202105','202205','合肥弘阳广场',0)
    form:0-所有业态；1-主次力店；2-一般商铺
    Return :
    项目名称	 平日销售日均	 周末销售日均	周末/平日比	周末占比同比
    合肥弘阳广场	788503.48	1123773.68  	0.58	-0.39
    """
    #导入数据并整理
    df_xs_zp = pd.read_excel(folderpath+shoppingmall+month1+'.xlsx')
    df_xs_dj = pd.read_excel(folderpath+shoppingmall+month2+'.xlsx')
    df_ppk = pd.read_excel('~/Desktop/销售客流pandas/品牌库.xlsx')
    df_xs_zp =df_xs_zp[['项目名称','铺位类型','品牌名称','营业额','营业额日期']]
    df_xs_dj =df_xs_dj[['项目名称','铺位类型','品牌名称','营业额','营业额日期']]
    df_xs_zp['营业额日期']=df_xs_zp.营业额日期.apply(lambda x:x.date())
    df_xs_dj['营业额日期']=df_xs_dj.营业额日期.apply(lambda x:x.date())

    df_ppk = df_ppk[['品牌名称','业态']]
    df_xs_all_ppk1 = pd.merge(df_xs_zp,df_ppk,on='品牌名称',how='left')
    df_xs_all_ppk2 = pd.merge(df_xs_dj,df_ppk,on='品牌名称',how='left')
    #根据业态，区分销售数据的统计范围
    if form==1:
        df_xs_all_ppk1=df_xs_all_ppk1.loc[(df_xs_all_ppk1.业态=='主次力店')]
        df_xs_all_ppk2=df_xs_all_ppk2.loc[(df_xs_all_ppk2.业态=='主次力店')]
    if form==2:
        df_xs_all_ppk1=df_xs_all_ppk1.loc[(df_xs_all_ppk1.业态!='主次力店')]
        df_xs_all_ppk2=df_xs_all_ppk2.loc[(df_xs_all_ppk2.业态!='主次力店')]

    #month1 周末平日销售结果
    df_xs_all_ppk1['周末'] = df_xs_all_ppk1['营业额日期'].apply(lambda x: '是' if x.weekday() >4 else '否')
    df_tmp1 = df_xs_all_ppk1.groupby(by='周末').agg({'营业额':'sum'}).reset_index()
    df_tmp1['日均营业额']=df_tmp1.apply(lambda x:x['营业额']/9 if x['周末']=='是' else x['营业额']/22,axis=1)

    a,b,c=df_tmp1.iat[0,2],df_tmp1.iat[1,2],df_tmp1.iat[1,1]/df_tmp1.iat[0,1]
    df_xs_res1 = pd.DataFrame(data={'项目名称':['合肥弘阳广场'],'平日销售日均':[a],'周末销售日均':[b],'周末/平日比':[c]})
    df1 = df_xs_res1.reset_index()

    #month2 周末平日销售结果
    df_xs_all_ppk2['周末'] = df_xs_all_ppk2['营业额日期'].apply(lambda x: '是' if x.weekday() >4 else '否')
    df_tmp2 = df_xs_all_ppk2.groupby(by='周末').agg({'营业额':'sum'}).reset_index()
    df_tmp2['日均营业额']=df_tmp2.apply(lambda x:x['营业额']/9 if x['周末']=='是' else x['营业额']/22,axis=1)

    a,b,c=df_tmp2.iat[0,2],df_tmp2.iat[1,2],df_tmp2.iat[1,1]/df_tmp2.iat[0,1]
    df_xs_res2 = pd.DataFrame(data={'项目名称':['合肥弘阳广场'],'平日销售日均':[a],'周末销售日均':[b],'周末/平日比':[c]})
    df2 = df_xs_res2.reset_index()

    #输出同比结果
    df2['上期周末占比']=df1.iat[0,4]
    df2['周末占比同比']=df2.iat[0,4]-df1.iat[0,4]

    return df2.round(2)

df_all = get_diff_xs('~/Desktop/销售客流pandas/','202105','202205','合肥弘阳广场',0)
df_zcld = get_diff_xs('~/Desktop/销售客流pandas/','202105','202205','合肥弘阳广场',1)
df_ybsp = get_diff_xs('~/Desktop/销售客流pandas/','202105','202205','合肥弘阳广场',2)

with pd.ExcelWriter('~/Downloads/xs_hefei.xlsx') as writer:
    df_all.to_excel(writer,sheet_name='所有业态')
    df_zcld.to_excel(writer,sheet_name='主次力店')
    df_ybsp.to_excel(writer,sheet_name='一般商铺')


In [ ]:
import sparklines

In [ ]:
import pandas as pd
import numpy as np

df_tj =pd.read_excel('~/Downloads/xs_hf_2205.xlsx',sheet_name='统计表',header=2)
df_hf_202105 =pd.read_excel('~/Downloads/xs_hf_2205.xlsx',sheet_name='主次力店202105')
df_hf_202205 =pd.read_excel('~/Downloads/xs_hf_2205.xlsx',sheet_name='主次力店202205')

df1=df_tj.iloc[:,1:]
df2=df_tj.iloc[:,:4]

with pd.ExcelWriter('~/Downloads/11111.xlsx') as writer:
    df1.to_excel(writer,sheet_name='s1')
    df2.to_excel(writer,sheet_name='s1',startrow=4)

In [ ]:
from part_analysis import get_part_analysis_blocklbs
import pandas as pd

dt_s,dt_e = '20220526','20220527'
df_s = get_part_analysis_blocklbs(df_stock_block,dt_s)
df_e = get_part_analysis_blocklbs(df_stock_block,dt_e)
df_s['日期'],df_e['日期']=dt_s,dt_e
df_r = pd.concat([df_s,df_e],keys=['行业编码','所属行业'])
df_r.to_excel('1234.xlsx')

In [ ]:
import pandas as pd
import requests
from datetime import datetime
from jsonpath import jsonpath


def get_zt_stock_rank(date: str = None) -> pd.DataFrame:
    """
    获取指定日期涨停股票行情

    Parameters
    ----------
    date : str
        指定日期，默认为当前日期 格式形如 ``'20220506'``

    Returns
    -------
    DataFrame
        指定日期涨停股票行情

    Fields
    ------
    ``['日期', '股票代码', '股票名称', '涨跌幅', '成交额', '封板资金', '流通市值', '最新价', '换手率','首次封板时间', '最后封板时间', '炸板次数', '连扳数', '所属行业']``
    """
    if date is None:
        date = datetime.today().strftime('%Y%m%d')
    params = (
        ('ut', '7eea3edcaed734bea9cbfc24409ed989'),
        ('dpt', 'wz.ztzt'),
        ('Pageindex', '0'),
        ('pagesize', '10000'),
        ('sort', 'fbt:asc'),
        ('date', date),
    )

    response = requests.get(
        'http://push2ex.eastmoney.com/getTopicZTPool', params=params,  verify=False)
    fields = {
        'c': '股票代码',
        'n': '股票名称',
        'zdp': '涨跌幅',
        'amount': '成交额',
        'fund': '封板资金',
        'ltsz': '流通市值',
        'p': '最新价',
        'hs': '换手率',
        'fbt': '首次封板时间',
        'lbt': '最后封板时间',
        'zbc': '炸板次数',
        'lbc': '连板数',
        'hybk': '所属行业'
    }
    items = jsonpath(response.json(), '$..pool[:]')
    if not items:
        df = pd.DataFrame(
            columns=['日期']+list(fields.values())+['统计天数', '涨停次数'])
        return df
    df = pd.DataFrame(items)
    extra_df: pd.DataFrame = pd.DataFrame.from_records(df['zttj']).rename(columns={
        'days': '统计天数',
        'ct': '涨停次数'
    })
    df: pd.DataFrame = pd.concat([df, extra_df], axis=1)
    df: pd.DataFrame = df.rename(columns=fields)[fields.values()]
    df['首次封板时间'] = df['首次封板时间'].apply(lambda x: pd.to_datetime(
        str(x), format='%H%M%S').strftime('%H:%M:%S'))
    df['最后封板时间'] = df['最后封板时间'].apply(lambda x: pd.to_datetime(
        str(x), format='%H%M%S').strftime('%H:%M:%S'))
    df['最新价'] /= 1000
    df['涨跌幅'] = df['涨跌幅'].apply(lambda x: round(x, 2))
    df.insert(0, '日期', date)

    # 修改列名，同同花顺数据
    col = {"涨跌幅":"涨跌幅"+date,"成交额":"成交额"+date,'封板资金':'封板资金'+date, '流通市值':'流通市值'+date, 
    '最新价':'最新价'+date, '换手率':'换手率'+date,'首次封板时间':'首次封板时间'+date, '最后封板时间':'最后封板时间'+date, '炸板次数':'炸板次数'+date, '连板数':'连板数'+date}

    df.rename(columns=col,inplace = True)
    return df

def grade(x,dt1,dt2):
    score = 0
    # if x['首次封板时间'] > x['最后封板时间']:
    #     score += 10
    # else:
    #     score -=10
    return score
dt1='20220524'
dt2 = '20220525' #datetime.today().strftime('%Y%m%d')

df1 = get_zt_stock_rank(dt1)
df2 = get_zt_stock_rank(dt2)

df3= pd.merge(df1,df2,on='股票代码',how='left')

df3['封板评分'+dt2]=df3.apply(lambda x:grade(x,dt1,dt2),axis=1)
# df4=df3.loc[('首次封板时间'+dt2)]

# 昨日涨停，今日连板
df3.dropna(inplace=True)
# df3=df3[df3['封板资金20220523'].notnull()]
# df3

# 昨日涨停，今日断板
df3= pd.merge(df1,df2,on='股票代码',how='left')
# df3=df3[df3['封板资金20220523'].isnull()]
# # 昨日连板3板以上，今日断板
# df3.loc[df3['连板数'+dt1] > 3]


# 哪些股票今日2进3，3进4等：where 今日连板=3 昨日连板10个，成功晋级4个，那数据在2->3 显示4/10
# 昨日2板，今日3板；
def up(x,dt1,dt2):
    if pd.isna(x['连板数'+dt2]):
        return '止步于:'+ str(x['连板数'+dt1]) +'板'
    else:
        return '晋级成功:'+str(x['连板数'+dt1])+'进'+str(x['连板数'+dt2])

df3.loc[:,'晋级']=df3.apply(lambda x:up(x,dt1,dt2),axis=1)
# df3.to_csv('20220524.csv',encoding='utf_8_sig')
# 首版晋级成功率
def compr(a,b,df,dt1,dt2):
    x = str(df.loc[(df['连板数'+dt1]==a)].count()['连板数'+dt1])
    y = str(df.loc[(df['连板数'+dt2]==b)].count()['连板数'+dt2])
    return y+'/'+x

i=1
while i<10:
    print('连板晋级成功率：'+str(i)+'进'+str(i+1)+':'+compr(i,i+1,df3,dt1,dt2))
    i +=1

# df5int=df1.loc[(df1.日期==dt1)&(df1.股票名称=='上海亚虹')]['炸板次数'+dt1]
# print(df5int)
# cat_count_series=df1['所属行业'].drop_duplicates()
# 实现 where in 用法 
cat_count_series=['专用设备','有色金属']
# df1.loc[(df1.所属行业.isin(cat_count_series))]

# 实现 where not in 用法 
# df1.loc[(-df1.所属行业.isin(cat_count_series))]

# df2.loc[(df2['连板数'+dt2]==1)].to_csv('20220525.csv',encoding='utf_8_sig')
# with pd.ExcelWriter('output.xlsx') as writer:
#     df1.to_excel(writer,sheet_name='sheetname-df1')
#     df2.to_excel(writer,sheet_name='sheetname-df2',startrow= 2,startcol=3)

df1



In [ ]:
from blockstocks import get_stocks_block
import urllib3

urllib3.disable_warnings()

df_stock_block = get_stocks_block()


In [ ]:
from dataclasses import replace
from datetime import datetime
from pydoc import describe
import pandas as pd
from pandas import DataFrame
from sqlalchemy import true
from get_stock_limit import get_stock_dbs_daily , get_stock_limitup_daily
from part_analysis import get_part_analysis_blocklbs,get_part_analysis_lbs_promotion
from output_style_format import to_excel_auto_column_weight
import urllib3

dt1,dt2,dt3,dt4,dt5='20220527','20220530','20220531','20220601','20220602'
urllib3.disable_warnings()
# df1 = get_part_analysis_lbs_promotion(df_stock_block,dt1)
# df2 = get_part_analysis_lbs_promotion(df_stock_block,dt2)
# df3 = get_part_analysis_lbs_promotion(df_stock_block,dt3)
# df4 = get_part_analysis_lbs_promotion(df_stock_block,dt4)
# df5 = get_part_analysis_lbs_promotion(df_stock_block,dt5)

# df_stock_block



# df_lbs = get_stock_limitup_daily(1,dt5)
# dfr = df_lbs.loc[(df_lbs['连板数']==1)]

# with pd.ExcelWriter('~/Downloads/涨停行情'+'20220602'+'.xlsx',engine='openpyxl') as writer:
#     # df1.to_excel(writer,sheet_name='连板行情'+dt1)
#     # df2.to_excel(writer,sheet_name='连板行情'+dt2)
#     # df3.to_excel(writer,sheet_name='连板行情'+dt3)
#     # df4.to_excel(writer,sheet_name='连板行情'+dt4)
#     df5.to_excel(writer,sheet_name='连板行情'+dt5)
#     to_excel_auto_column_weight(dfr,writer,'首板'+dt5)

# dfr['lb20220602'].astype(int)


# writer.save()
# df_lbs = get_part_analysis_blocklbs(df_stock_block,dt5)
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color:yellow' if v else '' for v in is_max]

df_limitup = get_stock_limitup_daily(1,'20220617').sort_values(by=['连板数'],ascending=False)
df_limitup['成交额'] = df_limitup['成交额']/100000000
df_limitup['封板资金'] = df_limitup['封板资金']/100000000
df_limitup['流通市值'] = df_limitup['流通市值']/100000000
df_limitup['封成比'] = df_limitup['封板资金'] / df_limitup['成交额']
df = df_limitup.round(1)
df = df.sort_values(by=['封板资金','成交额'],ascending=False)


import numpy as np
from openpyxl.utils import get_column_letter
import seaborn as sns
cm =  sns.light_palette('green',as_cmap=True)

def color_negative_red(val):
    color = 'gray' if val < 1 else 'red'
    return 'color: %s' % color

def makepretty(styler):
    styler.background_gradient(cmap=cm,subset=['流通市值','成交额','封板资金','最新价','换手率','连板数','炸板次数','封成比'])
    
    styler.applymap(color_negative_red,subset=['封板资金'])
    # styler.bar(subset=['封板资金','成交额'],color='#FFA500')
    styler.set_caption("今日涨停情况排行")
    styler.format({'成交额':'{:0}','封板资金':'{:0}','流通市值':'{:0}','最新价':'{:0}','换手率':'{:0}','封成比':'{:.1%}'})
    styler.hide_index()
    return styler

df.style.pipe(makepretty)


# with pd.ExcelWriter('~/Downloads/连板行情'+'20220617-2'+'.xlsx',engine='openpyxl') as writer:
#     df.style.pipe(makepretty).to_excel(writer,sheet_name='连板行情',engine='openpyxl')


In [ ]:
import pandas as pd
from stockcalendar import get_calendar_lastday

urllib3.disable_warnings()

end_dt = datetime.today().strftime('%Y%m%d')

dt_r,i,n,lbs = [],1,200,1

while i<n:
    dt_t = get_calendar_lastday('20220101',i,end_dt)
    dt_r.append(dt_t.replace('-',''))
    i += 1

dt_range = dt_r[::-1]
df_res = pd.DataFrame()
for dt in dt_range :
    df_limitup = get_stock_limitup_daily(lbs,dt)
    df_res = pd.concat([df_res,df_limitup])

# df = pd.read_excel('~/Downloads/data.xlsx',sheet_name='Sheet2')
df = df_res[['股票名称','日期','连板数']]
df1 = df.dropna()
df2 = df1.groupby(['日期','股票名称']).agg({'连板数':'mean'}).reset_index().sort_values(by=['股票名称','日期'])
# df2.groupby(['股票名称']).get_group('中通客车')
df3 =  df2.drop_duplicates(['股票名称'])
df4 = pd.DataFrame(columns = dt_range)
for i in df3['股票名称']:
    dft = df2.groupby(['股票名称']).get_group(i).T.reset_index().drop(1,axis=0)
    ar = np.array(dft)
    lst = ar.tolist()
    dft.columns = lst[0]
    dft['股票名称'] = i
    dft.reset_index().drop(index = 0,axis=0)
    df4 = pd.concat([dft,df4])

df5 = df4.fillna(0).loc[(df4.日期!='日期')]
df5.insert(0,'股票名称',df5.pop('股票名称'))
df6 = df5.reindex(sorted(df5.columns), axis=1)

df7 = df6.sort_values(by=dt_range[::-1],ascending=True)


import numpy as np
from openpyxl.utils import get_column_letter
import seaborn as sns
cm =  sns.light_palette('green',as_cmap=True)

def color_negative_red(val):
    color = 'gray' if val < 1 else 'red'
    return 'color: %s' % color

def makepretty(styler):
    styler.background_gradient(cmap=cm,subset=dt_range)
    
    c,i = [],0

    while i < len(dt_range):
        c.append('{:0}') 
        i += 1
    styler.format(dict(zip(dt_range,c)))
    # styler.set_caption("期间连板情况排行")
    # styler.hide_index()
    return styler

df8 = df7.drop(['日期'],axis=1).reset_index(drop=True)
df8['max'] = df8[dt_range].max(axis=1)
df9 =  df8.loc[(df8['max']>4)]

df9.style.pipe(makepretty)\
    .to_excel('~/Downloads/20220619-4-200.xlsx')

In [7]:


from part_analysis import get_stock_limitup_gantt
import urllib3
urllib3.disable_warnings()

days,lbs=10,6

style_df,df =  get_stock_limitup_gantt(days,lbs)
# style_df.to_excel('~/Downloads/20220619-400.xlsx')
style_df


,20220607,20220608,20220609,20220610,20220613,20220614,20220615,20220616,20220617,股票名称,max
165,4.0,5.0,6.0,7.0,0,0,0,0,0,华西能源,7.000000
214,1.0,2.0,3.0,4.0,5.0,0,0,0,0,科远智慧,5.000000
255,6.0,7.0,8.0,9.0,10.0,11.0,0,0,0,海汽集团,11.000000
300,0,0,1.0,2.0,3.0,4.0,5.0,0,0,郑州煤电,5.000000
421,0,0,0,0,1.0,2.0,3.0,4.0,5.0,浙江世宝,5.000000
422,0,0,0,1.0,2.0,3.0,4.0,5.0,6.0,集泰股份,6.000000
423,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,铖昌科技,10.000000


In [6]:
type(sty)

pandas.io.formats.style.Styler

# 使用例子


In [ ]:
import string
df = pd.DataFrame(np.random.rand(10,10)*1.2, columns=list(string.ascii_letters[:10]))
df


In [ ]:
df[(df>1).sum(1).gt(1)]

### 获取单只股票基本信息


In [ ]:
import efinance as ef
stock_code = '600519'
ef.stock.get_base_info(stock_code)


### 获取多只股票基本信息


In [ ]:
import efinance as ef

stock_codes = ['600519','300750']
ef.stock.get_base_info(stock_codes)
ef.stock.get_today_bill('600519')
ef.stock.get_deal_detail(stock_codes[0])

### 获取单只股票日 k 数据


In [ ]:

beg = '20200101' # 开始日期
end = '20210101'   # 结束日期
ef.stock.get_quote_history(stock_code,beg== beg,end=end)

#### 获取单只股票的 5 分钟 k 线数据


In [ ]:
import efinance as ef
stock_code = '600519'
beg = '20200101' # 开始日期
end = '20500101'   # 结束日期(最好比当前日期大，要不然可能没有数据)
klt = 5 # 指定获取的 k 线数据是 5 分钟的
ef.stock.get_quote_history(stock_code,beg = beg,end = end,klt=5)


### 获取多只股票日 k 数据(得到以股票代码为 key，以 pandas.DataFrame 为值的字典)


In [ ]:

beg = '20200101' # 开始日期
end = '20210101'   # 结束日期
stock_codes = ['600519','300750']
df_items = ef.stock.get_quote_history(stock_codes,beg== beg,end=end)

In [ ]:
type(df_items) # 查看类型

In [ ]:
df_items.keys()

In [ ]:
df_items[stock_codes[0]] # 查看第一支股票的的 k 线数据

### 沪深市场股票最新季度业绩表现


In [ ]:
import efinance as ef
ef.stock.get_all_company_performance()

### 沪深市场股票指定日期的季度业绩表现


In [ ]:
import efinance as ef
ef.stock.get_all_company_performance(date='2021-03-31')

### 股票最新交易日单子流入情况(分钟级)


In [ ]:
import efinance as ef
ef.stock.get_today_bill('600519')

### 股票历史交易日单子流入情况(日级)


In [ ]:
import efinance as ef
ef.stock.get_history_bill('600519')

### 股票最新龙虎榜


In [ ]:
ef.stock.get_daily_billboard()

### 指定日期区间的股票龙虎榜


In [ ]:
# 开始日期
start_date = '2021-08-20'
# 结束日期
end_date = '2021-08-27'
ef.stock.get_daily_billboard(start_date=start_date,
                             end_date=end_date)


### 股东数目


In [ ]:
ef.stock.get_latest_holder_number()

# 更多使用例子详见文档 [efinance-docs](https://efinance.readthedocs.io/)
